<a href="https://colab.research.google.com/github/Chohyungun/Chohyungun/blob/main/Catboost%EC%97%B0%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 최종 결과물 : Catboost(Public : 5.1721157857, Private : 8.4329062881)


## 1.Library & Data Load

In [1]:
from google.colab import drive
drive.mount('/content/drive')      

Mounted at /content/drive


In [2]:
!pip install catboost
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 1.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 308 kB 5.1 MB/s 
     |████████████████████████████████| 81 kB 9.9 MB/s 
     |████████████████████████████████| 209 kB 47.3 MB/s 
     |████████████████████████████████| 78 kB 6.6 MB/s 
     |████████████████████████████████| 146 kB 51.1 MB/s 
     |████████████████████████████████| 49 kB 5.9 MB/s 
     |████████████████████████████████| 112 kB 56.6 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=e2598b1efb537b49ec89ede6a329651741fd28454e79f5b530b0c3547264ee01
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [3]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor

from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold,train_test_split
from sklearn.metrics import mean_squared_error

import random
import optuna
from optuna.samplers import TPESampler

In [4]:
path='/data/235867_╣░╖∙ └п┼ы╖о ┐╣├° ░ц┴°┤ы╚╕/'

In [5]:
train = pd.read_csv(path+'train_df.csv', encoding='CP949')
test = pd.read_csv(path+'test_df.csv', encoding='CP949')

submission = pd.read_csv(path+'sample_submission.csv')

FileNotFoundError: ignored

## 2. Data Preprocess

송하인 격자공간고유번호와 수하인격자공간고유번호가 연속형 자료로 취급하면 안된다고 생각하였습니다.

격자공간고유번호의 16자리 중 자릿수마다 담고 있는 정보가 다를 것이라 생각하였고,
데이터를 탐색해본 결과 1 ~ 5자리, 6 ~ 9자리, 10, 11 ~ 16자리가 가지고 있는 정보가 다를 것이라 생각했습니다.

그리고 송하인 격자공간고유번호의 unique 수는 꽤 되지만 수하인 격자공간고유번호의 unique 수는 얼마 되지 않았습니다. 

따라서 송하인 격자공간고유번호는 1 ~ 5, 6 ~ 9, 10, 11 ~ 16자릿수로 나누고
수하인 격자공간고유번호는 자릿수 별로 변수를 생성하였습니다.

총 22개의 설명변수로 이루어진 데이터로 변환하였습니다.

In [ ]:
def numround(number, digit):
  num=[]
  while(number!=0):
    num.append(number % 10)
    number = number //10

  return int(num[-digit])

In [ ]:
for i in tqdm(range(16)):
  train[f'SEND_SPG_INNB_{i+1}'] = 0
  train[f'REC_SPG_INNB_{i+1}'] = 0
  test[f'SEND_SPG_INNB_{i+1}'] = 0
  test[f'REC_SPG_INNB_{i+1}'] = 0
  for j in range(train.shape[0]):
    train.loc[j,f'SEND_SPG_INNB_{i+1}']=numround(train.loc[j,'SEND_SPG_INNB'],i+1)
    train.loc[j,f'REC_SPG_INNB_{i+1}']=numround(train.loc[j,'REC_SPG_INNB'],i+1)

  for j in range(test.shape[0]):
    test.loc[j,f'SEND_SPG_INNB_{i+1}']=numround(test.loc[j,'SEND_SPG_INNB'],i+1)
    test.loc[j,f'REC_SPG_INNB_{i+1}']=numround(test.loc[j,'REC_SPG_INNB'],i+1)


In [ ]:
train['SEND_SPG_INNB_1~5']=train['SEND_SPG_INNB_1']+train['SEND_SPG_INNB_2']+train['SEND_SPG_INNB_3']+train['SEND_SPG_INNB_4']+train['SEND_SPG_INNB_5']
train['SEND_SPG_INNB_6~9']=train['SEND_SPG_INNB_6']+train['SEND_SPG_INNB_7']+train['SEND_SPG_INNB_8']+train['SEND_SPG_INNB_9']
train['SEND_SPG_INNB_10']=train['SEND_SPG_INNB_10']
train['SEND_SPG_INNB_11~16']=train['SEND_SPG_INNB_11']+train['SEND_SPG_INNB_12']+train['SEND_SPG_INNB_13']+train['SEND_SPG_INNB_14']+train['SEND_SPG_INNB_15']+train['SEND_SPG_INNB_16']

test['SEND_SPG_INNB_1~5']=test['SEND_SPG_INNB_1']+test['SEND_SPG_INNB_2']+test['SEND_SPG_INNB_3']+test['SEND_SPG_INNB_4']+test['SEND_SPG_INNB_5']
test['SEND_SPG_INNB_6~9']=test['SEND_SPG_INNB_6']+test['SEND_SPG_INNB_7']+test['SEND_SPG_INNB_8']+test['SEND_SPG_INNB_9']
test['SEND_SPG_INNB_10']=test['SEND_SPG_INNB_10']
test['SEND_SPG_INNB_11~16']=test['SEND_SPG_INNB_11']+test['SEND_SPG_INNB_12']+test['SEND_SPG_INNB_13']+test['SEND_SPG_INNB_14']+test['SEND_SPG_INNB_15']+test['SEND_SPG_INNB_16']


In [ ]:
train.index=train['index']
test.index=test['index']
train.drop(['REC_SPG_INNB','SEND_SPG_INNB','SEND_SPG_INNB_1','SEND_SPG_INNB_2','SEND_SPG_INNB_3','SEND_SPG_INNB_4','SEND_SPG_INNB_5','SEND_SPG_INNB_6','SEND_SPG_INNB_7',
            'SEND_SPG_INNB_8','SEND_SPG_INNB_9','SEND_SPG_INNB_11','SEND_SPG_INNB_12','SEND_SPG_INNB_13','SEND_SPG_INNB_14','SEND_SPG_INNB_15','SEND_SPG_INNB_16','index'],axis=1,inplace=True)
test.drop(['REC_SPG_INNB','SEND_SPG_INNB','SEND_SPG_INNB_1','SEND_SPG_INNB_2','SEND_SPG_INNB_3','SEND_SPG_INNB_4','SEND_SPG_INNB_5','SEND_SPG_INNB_6','SEND_SPG_INNB_7',
            'SEND_SPG_INNB_8','SEND_SPG_INNB_9','SEND_SPG_INNB_11','SEND_SPG_INNB_12','SEND_SPG_INNB_13','SEND_SPG_INNB_14','SEND_SPG_INNB_15','SEND_SPG_INNB_16','index'],axis=1,inplace=True)

In [ ]:
for col in test.columns:
  train[col]=train[col].astype('category')
  test[col]=test[col].astype('category')

In [ ]:
#Optuna용 Train셋
X = train.drop(['INVC_CONT'],axis=1)
y = train['INVC_CONT']
X_test = test.copy()

In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
X_test.head()

In [ ]:
train.info()

## 3. 모델링

Optuna로 Random Search를 통해 Catboost 최적의 파라미터를 사용하였습니다.
objective 함수의 param에 파라미터를 넣고, 구간을 넣으면 랜덤한 값으로 학습되며
rmse값이 반환되는 함수입니다. "trial"에 반복 횟수를 작성하면 됩니다.

Catboost 특성상 학습이 오래 걸리기 때문에 최적의 파라미터를 찾아 cat_param로 정의하였습니다.

(아래코드는 AIBoo님의 신용카드 사용자 연체 예측 AI 경진대회 [Private 8위 0.66203] | TYKIM | Catboost 코드를 참고하여 수정하였습니다.)

In [ ]:
# def objective(trial):
#   param = {
#       "random_state":42,
#       'learning_rate' : trial.suggest_loguniform('learning_rate', 0.01, 0.05),
#       'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 0.01, 100.00),
#       "n_estimators":trial.suggest_int("n_estimators", 500, 5000),
#       "max_depth":trial.suggest_int("max_depth", 4, 16),
#       'random_strength' :trial.suggest_int('random_strength', 0, 100),
#       "colsample_bylevel":trial.suggest_float("colsample_bylevel", 0.4, 1.0),
#       "l2_leaf_reg":trial.suggest_float("l2_leaf_reg",1e-8,3e-5),
#       "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
#       "max_bin": trial.suggest_int("max_bin", 200, 500),
#       'od_type': trial.suggest_categorical('od_type', ['IncToDec', 'Iter']),
#   }
#   X_train, X_valid, y_train, y_valid = train_test_split(X,y,test_size=0.2)
#   cat_features = range(X_test.shape[1])
#   cat = CatBoostRegressor(**param)
#   cat.fit(X_train, y_train,
#           eval_set=[(X_train, y_train), (X_valid,y_valid)],
#           early_stopping_rounds=35,cat_features=cat_features,
#           verbose=100)
#   cat_pred = cat.predict(X_valid)
#   rmse = np.sqrt(mean_squared_error(y_valid, cat_pred))

#   return rmse

In [ ]:
# sampler = TPESampler(seed=42)
# study = optuna.create_study(
#     study_name = 'cat_parameter_opt',
#     direction = 'minimize',
#     sampler = sampler,
# )

# study.optimize(objective, n_trials=10)
# print("Best Score:",study.best_value)
# print("Best trial",study.best_trial.params)

In [ ]:
cat_param={'learning_rate': 0.018272261776066247, 
           'bagging_temperature': 63.512210106407046, 
           'n_estimators': 3794, 
           'max_depth': 11, 
           'random_strength': 15, 
           'colsample_bylevel': 0.49359671220172163, 
           'l2_leaf_reg': 1.7519275289243016e-06, 
           'min_child_samples': 88, 
           'max_bin': 380, 
           'od_type': 'IncToDec'
           }


In [ ]:
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
folds = []
for train_idx, valid_idx in skf.split(train, train['INVC_CONT']):
  folds.append((train_idx,valid_idx))

train 데이터를 K-Fold 하여 각 Fold의 학습 값을 가지고 test 예측값을 구한 후
평균을 구하였습니다.

In [ ]:
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
folds = []
for train_idx, valid_idx in skf.split(train, train['INVC_CONT']):
  folds.append((train_idx,valid_idx))

random.seed(42)
cat_models={}

cat_features =range(X_test.shape[1])

for fold in range(10):
  print(f'===================================={fold+1}============================================')
  train_idx, valid_idx = folds[fold]
  X_train = train.drop(['INVC_CONT'],axis=1).iloc[train_idx]
  X_valid = train.drop(['INVC_CONT'],axis=1).iloc[valid_idx]
  y_train = train['INVC_CONT'][train_idx].values
  y_valid = train['INVC_CONT'][valid_idx].values

  cat = CatBoostRegressor(**cat_param)
  cat.fit(X_train, y_train,
          eval_set=[(X_train, y_train), (X_valid,y_valid)],
          early_stopping_rounds=35,cat_features=cat_features,
          verbose=100)
  cat_models[fold] = cat
  print(f'================================================================================\n\n')

In [ ]:
submission.loc[:,'INVC_CONT']=0
for fold in range(10):
  submission.loc[:,'INVC_CONT'] += cat_models[fold].predict(test)/10

train을 K-fold한 값의 평균을 구하다 보니 예측값의 극단값이 작아질 수 밖에 없습니다.

따라서 INVC_CONT가 30 이상인 값에 적당한 값을 곱하여서 조정하였습니다.

In [ ]:
submission.loc[submission.INVC_CONT>30,'INVC_CONT']=submission.loc[submission.INVC_CONT>30,'INVC_CONT']*4.8
submission.to_csv('/data/submission/마지막 확인용.csv',index = False)